## Scoring an OPL model and space data assets with Watson Machine Learning

This notebook shows you how to deploy an OPL model, use data from the deployment space, and get solutions using the Watson Machine Learning Python Client.

This notebook runs on Python.

**Table of contents:**

- [Set up the Watson Machine Learning client](#setup)
- [Create a client instance](#create)
- [Prepare your model archive](#prepare)
- [Upload your model on Watson Machine Learning](#upload)
- [Create a deployment](#deploy)
- [Create and monitor a job with space data asset for your deployed model](#job)
- [Summary](#summary)

<a id='setup'></a>
### Set up the Watson Machine Learning client

Install and then import the Watson Machine Learning client library.

In [ ]:
# Install WML client API

!pip install ibm-watson-machine-learning

In [ ]:
from ibm_watson_machine_learning import APIClient

### Pre-requisite
As a prerequisite to running this sample, you must put the data in the space repository.

In a real application, the space repository will be updated by a 3rd party application.

The aim of this notebook is not to demonstrate how to update such a remote repository, but to show you how to connect an optimization model to space data assets.

Consequently the space repository is updated here manually.

The steps are as follows:
* Store the following .dat file on your local laptop as a model.dat file.
* Drag and drop the file into the space using the action "Drop files here or browse for files to upload".
* Navigate to the data description, then copy the ID and paste it here as space_data_id.

```
Products = {<"kluski", 100, 0.6, 0.8>, <"capellini", 200,0.8, 0.9>,<"fettucine", 300, 0.3, 0.4>}; 
Resources = {<"flour", 20>,<"eggs", 40>};
Consumptions = {<"kluski", "flour", 0.5>,<"kluski", "eggs", 0.2>,<"capellini", "flour", 0.4>,<"capellini", "eggs", 0.4>,<"fettucine", "flour", 0.3>,<"fettucine", "eggs", 0.6>};
```


In [ ]:
## Provide the following properties
space_data_id = "<space_data_id>"

<a id='create'></a>
### Create a client instance

In [ ]:
# Instantiate a client using credentials

cluster = "<your_cluster>"
username = "<username>"
password = "<password>"

wml_credentials = {
    "username": username,
    "password": password,
    "instance_id" : "wml_local",
    "url": cluster,
    "version": "4.8"
}

client = APIClient(wml_credentials)

In [ ]:
client.version

<a id='prepare'></a>
### Prepare your model archive

Use the `write_file` command to write these models to a `model.mod` file. 

Use the `tar` command to create a tar archive.

In [ ]:
%mkdir model

In [ ]:
%%writefile model.mod

tuple TProduct {
  key string name;
  float demand;
  float insideCost;
  float outsideCost;
};

tuple TResource {
  key string name;
  float capacity;
};

tuple TConsumption {
  key string productId;
  key string resourceId;
  float consumption; 
}

{TProduct}     Products = ...;
{TResource}    Resources = ...;
{TConsumption} Consumptions = ...;

/// solution
tuple TPlannedProduction {
  key string productId;
  float insideProduction;
  float outsideProduction; 
}

/// variables.
dvar float+ Inside [Products];
dvar float+ Outside[Products];

dexpr float totalInsideCost  = sum(p in Products)  p.insideCost * Inside[p];
dexpr float totalOutsideCost = sum(p in Products)  p.outsideCost * Outside[p];

minimize
  totalInsideCost + totalOutsideCost;
   
subject to {
  forall( r in Resources )
    ctCapacity: 
      sum( k in Consumptions, p in Products 
           : k.resourceId == r.name && p.name == k.productId ) 
        k.consumption* Inside[p] <= r.capacity;

  forall(p in Products)
    ctDemand:
      Inside[p] + Outside[p] >= p.demand;
}

{TPlannedProduction} plan = {<p.name, Inside[p], Outside[p]> | p in Products};

// Display the production plann
execute DISPLAY_PLAN {
  for( var p in plan ) {
    writeln("p[",p.productId,"] = ",p.insideProduction," inside, ", p.outsideProduction, " outside.");
  }
}

In [ ]:
import tarfile
def reset(tarinfo):
    tarinfo.uid = tarinfo.gid = 0
    tarinfo.uname = tarinfo.gname = "root"
    return tarinfo
tar = tarfile.open("model.tar.gz", "w:gz")
tar.add("model.mod", arcname="model.mod", filter=reset)
tar.close()

<a id='upload'></a>
### Upload your model on Watson Machine Learning

Store model in Watson Machine Learning with:
* the tar archive previously created,
* metadata including the model type and runtime

Get the `model_uid`.

In [ ]:
# Find the space ID

space_name = "<space_name>"

space_id = [x['metadata']['id'] for x in client.spaces.get_details()['resources'] if x['entity']['name'] == space_name][0]

client.set.default_space(space_id)

In [ ]:
mnist_metadata = {
    client.repository.ModelMetaNames.NAME: "PastaProduction",
    client.repository.ModelMetaNames.DESCRIPTION: "Model for OPL PastaProduction",
    client.repository.ModelMetaNames.TYPE: "do-opl_22.1",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: client.software_specifications.get_uid_by_name("do_22.1"),
}

model_details = client.repository.store_model(model='/home/wsuser/work/model.tar.gz', meta_props=mnist_metadata)

model_uid = client.repository.get_model_id(model_details)

<a id='deploy'></a>
### Create a deployment 

Create a batch deployment for the model, providing information such as:
* the maximum number of compute nodes
* the T-shirt size of the compute nodes

Get the `deployment_uid`.

In [ ]:
meta_props = {
    client.deployments.ConfigurationMetaNames.NAME: "PastaProduction Deployment",
    client.deployments.ConfigurationMetaNames.DESCRIPTION: "PastaProduction Deployment",
    client.deployments.ConfigurationMetaNames.BATCH: {},
    client.deployments.ConfigurationMetaNames.HARDWARE_SPEC: {'name': 'S', 'num_nodes': 1}
}

deployment_details = client.deployments.create(model_uid, meta_props=meta_props)

deployment_uid = client.deployments.get_uid(deployment_details)

# print deployment id if needed
# print( deployment_uid )

In [ ]:
# List all existing deployments

client.deployments.list()

<a id='job'></a>
### Create and monitor a job with space data asset for your deployed model

Create a payload containing input data from the space asset.

Create a new job with this payload and the deployment.

Get the `job_uid`.

In [ ]:
space_id

In [ ]:
#{"connection":{},"id":"id4W4WMZG73LB","location":{"description":"","name":"Output Batch deployment"},"type":"data_asset"}
data = {"type" : "data_asset", 
        "id" : "model.dat", 
        "location" : { 
            "href":"/v2/assets/"+space_data_id+"?space_id="+space_id
        },
        "connection" : {
        }
       }
print(data)

In [ ]:
#Here we use INPUT_DATA_REFERENCES instead of INPUT_DATA as this is referenced data
solve_payload = {
    "solve_parameters" : {
        "oaas.logAttachmentName":"log.txt",
        "oaas.logTailEnabled":"true",
        "oaas.resultsFormat": "csv"
    },
    client.deployments.DecisionOptimizationMetaNames.INPUT_DATA_REFERENCES: [
        data
    ],
    client.deployments.DecisionOptimizationMetaNames.OUTPUT_DATA: [
        {
            "id":".*\.csv"
        }
    ]
}
job_details = client.deployments.create_job(deployment_uid, solve_payload)
job_uid = client.deployments.get_job_uid(job_details)

Display job status until it is completed.

The first job of a new deployment might take some time as a compute node must be started.

In [ ]:
from time import sleep

time_spent = 0

while time_spent < 120 and job_details['entity']['decision_optimization']['status']['state'] not in ['completed', 'failed', 'canceled']:
    print(job_details['entity']['decision_optimization']['status']['state'] + '...')
    sleep(5)
    time_spent+=5
    job_details=client.deployments.get_job_details(job_uid)

#print( job_details['entity']['decision_optimization'])

In [ ]:
if job_details['entity']['decision_optimization']['status']['state'] == "running":
    print(job_details)
    client.deployments.delete_job(job_uid)
    print("Something went wrong")

In [ ]:
print(job_details['entity']['decision_optimization']['solve_state']['solve_status'])

### Delete the deployment

Use the following method to delete the deployment.

In [ ]:
client.deployments.delete(deployment_uid)

<a id='summary'></a>
### Summary and next steps

You've successfully completed this notebook! 

You've learned how to:

- work with the Watson Machine Learning client
- prepare your model archive and upload your model on Watson Machine Learning
- create a deployment
- create and monitor a job with connected data for your deployed model

Note: you can also use connected data for outputs.

Check out our online documentation <a href="https://www.ibm.com/docs/en/cloud-paks/cp-data/4.8.x" target="_blank" rel="noopener noreferrer">here</a> for more samples, tutorials and documentation.

<hr>
Copyright © 2019-2023. This notebook and its source code are released under the terms of the MIT License.